<a href="https://colab.research.google.com/github/abhinavarorags/CoolStuff/blob/test/QuantHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("stuff")

stuff


In [24]:
import pandas as pd
url = 'https://raw.githubusercontent.com/abhinavarorags/CoolStuff/refs/heads/test/s.csv'
url = 'https://raw.githubusercontent.com/abhinavarorags/CoolStuff/refs/heads/test/sample_data.csv'

try:
  df = pd.read_csv(url, sep=',')
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")
df.head()
print(f"Len: {len(df)}, Shape: {df.shape[0]}, Index Size: {df.index.size}, Size/Columns: {df.size // df.columns.size}")


Len: 67048, Shape: 67048, Index Size: 67048, Size/Columns: 67048


In [40]:
df_training = df.iloc[:20000]  # First 100 rows
train_data = df_training
df_test = df.iloc[20001:]     # Rows from index 100 onwards
test_data = df_test.copy()

In [ ]:
#DO NOT RUN THIS CELL
#This is the REAL Q-learning code but it runs slower
#Also needs lots of GPU power
#See image asking for 84 hours of runtime
#https://github.com/abhinavarorags/CoolStuff/blob/test/QuantFury%20Q-learning%20Compute.png
import numpy as np
import pandas as pd
import random

# Using the previously loaded train_data from '/mnt/data/training.csv'

# Q-learning parameters
initial_alpha = 0.5  # Initial learning rate
min_alpha = 0.01  # Minimum learning rate
alpha_decay = 0.995  # Decay rate for alpha

initial_epsilon = 1.0  # Initial exploration rate
min_epsilon = 0.01  # Minimum exploration rate
epsilon_decay = 0.995  # Decay rate for epsilon

gamma = 0.95  # Discount factor
num_episodes = 1000  # Number of episodes

# Define actions
ACTIONS = {0: 'Buy Long', 1: 'Sell Short', 2: 'Hold'}

# Initialize Q-table
q_table = np.zeros((len(train_data), len(ACTIONS)))

# Function to choose an action using epsilon-greedy strategy
def choose_action(state_index, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.choice(list(ACTIONS.keys()))  # Explore
    else:
        return np.argmax(q_table[state_index])  # Exploit

# Q-learning process
for episode in range(num_episodes):
    alpha = max(min_alpha, initial_alpha * (alpha_decay ** episode))
    epsilon = max(min_epsilon, initial_epsilon * (epsilon_decay ** episode))
    total_reward = 0

    for state_index in range(len(train_data) - 1):
        current_state = train_data.iloc[state_index].values[:-1]  # Exclude Date
        action = choose_action(state_index, epsilon)
        reward = train_data.iloc[state_index + 1]['Close'] - train_data.iloc[state_index]['Close']
        next_state_index = state_index + 1
        best_future_q = np.max(q_table[next_state_index]) if next_state_index < len(train_data) else 0
        q_table[state_index, action] += alpha * (reward + gamma * best_future_q - q_table[state_index, action])
        total_reward += reward

    if episode % 100 == 0:
        print(f"Episode: {episode}, Alpha: {alpha:.4f}, Epsilon: {epsilon:.4f}, Total Reward: {total_reward}")

# Save the Q-table to CSV
q_table_df = pd.DataFrame(q_table, columns=ACTIONS.keys())
#q_table_df.to_csv('/mnt/data/q_table.csv', index=False)
print("Q-table training complete and saved as q_table.csv.")


In [34]:
#Dumbed down the code to only 'Close' Prices and 10 episodes
print("Running Code on Training Data")

import numpy as np
import pandas as pd
import random

# Using the previously loaded train_data from '/mnt/data/training.csv'

# Q-learning parameters
initial_alpha = 0.5  # Initial learning rate
min_alpha = 0.01  # Minimum learning rate
alpha_decay = 0.995  # Decay rate for alpha

initial_epsilon = 1.0  # Initial exploration rate
min_epsilon = 0.01  # Minimum exploration rate
epsilon_decay = 0.995  # Decay rate for epsilon

gamma = 0.95  # Discount factor

# Convert necessary columns to NumPy arrays for faster access
close_prices = train_data['Close'].values
actions = np.array(list(ACTIONS.keys()))

# Reduce the number of episodes for demonstration
num_episodes = 10  # Use a smaller number for quicker demonstration

# Initialize Q-table
q_table = np.zeros((len(close_prices) - 1, len(actions)))

# Run the Q-learning algorithm with optimized data handling
for episode in range(num_episodes):
    alpha = max(min_alpha, initial_alpha * (alpha_decay ** episode))
    epsilon = max(min_epsilon, initial_epsilon * (epsilon_decay ** episode))
    total_reward = 0

    for i in range(len(close_prices) - 1):
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)
        else:
            action = actions[np.argmax(q_table[i])]

        reward = close_prices[i + 1] - close_prices[i]
        next_state_action_values = q_table[i + 1] if i + 1 < len(close_prices) - 1 else np.zeros(len(actions))
        best_future_q = np.max(next_state_action_values)
        q_table[i, action] += alpha * (reward + gamma * best_future_q - q_table[i, action])
        total_reward += reward

    print(f"Episode: {episode}, Alpha: {alpha:.4f}, Epsilon: {epsilon:.4f}, Total Reward: {total_reward}")

# Convert Q-table to DataFrame and save to CSV
q_table_df = pd.DataFrame(q_table, columns=[str(action) for action in actions])
#q_table_df.to_csv('/mnt/data/q_table_optimized.csv', index=False)
print("Optimized Q-table training complete and saved.")


Running Code on Training Data
Episode: 0, Alpha: 0.5000, Epsilon: 1.0000, Total Reward: 173.41
Episode: 1, Alpha: 0.4975, Epsilon: 0.9950, Total Reward: 173.41
Episode: 2, Alpha: 0.4950, Epsilon: 0.9900, Total Reward: 173.41
Episode: 3, Alpha: 0.4925, Epsilon: 0.9851, Total Reward: 173.41
Episode: 4, Alpha: 0.4901, Epsilon: 0.9801, Total Reward: 173.41
Episode: 5, Alpha: 0.4876, Epsilon: 0.9752, Total Reward: 173.41
Episode: 6, Alpha: 0.4852, Epsilon: 0.9704, Total Reward: 173.41
Episode: 7, Alpha: 0.4828, Epsilon: 0.9655, Total Reward: 173.41
Episode: 8, Alpha: 0.4803, Epsilon: 0.9607, Total Reward: 173.41
Episode: 9, Alpha: 0.4779, Epsilon: 0.9559, Total Reward: 173.41
Optimized Q-table training complete and saved.


In [41]:
q_table=q_table_df #Reassigned from pandas data frame to numpy array Hack
print("Running Code on Test Data")

# Extracting Close prices from the test data for action determination
close_prices_test = test_data['Close'].values

# Preparing the output DataFrame
recommended_actions = []

# Determine the best action for each state based on the Q-table
for i in range(len(close_prices_test) - 1):
    state_index = i % len(q_table)  # Looping through Q-table if test data is longer than Q-table
    action_index = np.argmax(q_table.iloc[state_index])
    recommended_actions.append(ACTIONS[action_index])

# Adding recommended actions to the test data
test_data['Recommended Action'] = recommended_actions + ['N/A']  # Last entry has no subsequent state

# Save the results with recommendations
#test_data.to_csv('/mnt/data/test_with_actions.csv', index=False)
print("Test Data saved with recommendations")


Running Code on Test Data
Test Data saved with recommendations


In [42]:
test_data.iloc[:5]

,Open,High,Low,Close,LongVolume,ShortVolume,LongCount,ShortCount,LongAvgPnlOpenSystem,ShortAvgPnlOpenSystem,LongPercentAveragePositionMaxTradingPower,ShortPercentAveragePositionMaxTradingPower,Date,Recommended Action
20001,242.80,243.20,240.10,240.92,197072.56,481.84,237,2,0.055013,0.024508,0.089598,0.120460,2021-07-27 13:35:00.000,Sell Short
20002,241.23,243.21,240.30,243.00,199503.00,486.00,239,2,0.063471,0.016086,0.091005,0.121500,2021-07-27 13:40:00.000,Buy Long
20003,243.10,243.20,241.80,242.25,198160.50,969.00,239,3,0.060177,0.010546,0.090825,0.107667,2021-07-27 13:45:00.000,Sell Short
20004,242.03,242.30,238.00,238.85,203022.50,716.55,232,2,0.045526,0.018746,0.085138,0.099521,2021-07-27 13:50:00.000,Sell Short
20005,238.70,240.30,238.53,239.85,205791.30,0.00,235,0,0.049257,0.000000,0.086674,0.000000,2021-07-27 13:55:00.000,Sell Short
